System information (for reproducibility):

In [1]:
versioninfo()

Julia Version 1.9.0
Commit 8e63055292 (2023-05-07 11:25 UTC)
Platform Info:
  OS: Windows (x86_64-w64-mingw32)
  CPU: 16 × AMD Ryzen 7 5800 8-Core Processor              
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-14.0.6 (ORCJIT, znver3)
  Threads: 9 on 16 virtual cores
Environment:
  JULIA_NUM_THREADS = 8


Load packages:

In [2]:
using Pkg

Pkg.activate(pwd())
Pkg.instantiate()
Pkg.status()

using DataFrames, Convex, MosekTools, MathOptInterface 

  Activating project at `C:\Users\larry\Dropbox\zza\UCLA\academic\year 3\quarter 3\257\biostat-257-2023-spring\hw5`


Status `C:\Users\larry\Dropbox\zza\UCLA\academic\year 3\quarter 3\257\biostat-257-2023-spring\hw5\Project.toml`
  [f65535da] Convex v0.15.3
  [a93c6f00] DataFrames v1.5.0
  [87dc4568] HiGHS v1.5.1
  [b99e6be6] Hypatia v0.7.3
  [4076af6c] JuMP v1.11.1
  [b8f27783] MathOptInterface v1.16.0
  [6405355b] Mosek v10.0.2
  [1ec41992] MosekTools v0.15.0
  [2f354839] Pajarito v0.8.2
  [08abe8d2] PrettyTables v2.2.4
  [c946c3f1] SCS v1.1.4
  [3eaba693] StatsModels v0.7.2


In this exercise, we practice using disciplined convex programming (SDP in particular) to solve optimal design problems.

## Introduction to Optimal design

Consider a linear model
\begin{eqnarray*}
	y_i = \mathbf{x}_i^T \boldsymbol{\beta} + \epsilon_i, \quad i = 1,\ldots, n,
\end{eqnarray*}
where $\epsilon_i$ are independent Gaussian noises with common variance $\sigma^2$. It is well known that the least squares estimate $\hat{\boldsymbol{\beta}}$ is unbiased and has covariance $\sigma^2 (\sum_{i=1}^n \mathbf{x}_i \mathbf{x}_i^T)^{-1}$. 

In **exact optimal design**, given total number of $n$ allowable experiments, we want to choose among a list of $m$ candidate design points $\{\mathbf{x}_1, \ldots, \mathbf{x}_m\}$ such that the covariance matrix is minimized in some sense. In mathematical terms, we want to find an integer vector $\mathbf{n} = (n_1, \ldots, n_m)$ such that $n_i \ge 0$, $\sum_{i=1}^m n_i = n$, and the matrix $\mathbf{V} = \left( \sum_{i=1}^m n_i \mathbf{x}_i \mathbf{x}_i^T \right)^{-1}$ is "small".

In **approximate optimal design**,  we want to find a probability vector $\mathbf{p} = (p_1, \ldots, p_m)$ such that $p_i \ge 0$, $\sum_{i=1}^m p_i = 1$, and the matrix $\mathbf{V} = \left( \sum_{i=1}^m p_i \mathbf{x}_i \mathbf{x}_i^T \right)^{-1}$ is "small".

Commonly used optimal design criteria include:

- In **$D$-optimal design**, we minimize the determinant of $\mathbf{V}$
\begin{eqnarray*}
	&\text{minimize}& \det \left( \sum_{i=1}^m p_i \mathbf{x}_i \mathbf{x}_i^T \right)^{-1} \\
	&\text{subject to}& p_i \ge 0, \sum_{i=1}^m p_i = 1.
\end{eqnarray*}

- In **$E$-optimal design**, we minimize the spectral norm, i.e., the maximum eigenvalue of $\mathbf{V}$
\begin{eqnarray*}
	&\text{minimize}& \lambda_{\text{max}} \left( \sum_{i=1}^m p_i \mathbf{x}_i \mathbf{x}_i^T \right)^{-1} \\
	&\text{subject to}& p_i \ge 0, \sum_{i=1}^m p_i = 1.	
\end{eqnarray*}
Statistically we are minimizing the maximum variance of $\sum_{j=1}^p a_j \text{var}(\hat \beta_j)$ over all vectors $\mathbf{a}$ with unit norm.

- In **$A$-optimal design**, we minimize the trace of $\mathbf{V}$
\begin{eqnarray*}
	&\text{minimize}& \text{tr} \left( \sum_{i=1}^m p_i \mathbf{x}_i \mathbf{x}_i^T \right)^{-1} \\
	&\text{subject to}& p_i \ge 0, \sum_{i=1}^m p_i = 1.
\end{eqnarray*}
Statistically we are minimizing the total variance $\sum_{j=1}^p \text{var}(\hat \beta_j)$.

## Q1 (10 pts) 3x4 factorial design

A drug company ask you to help design a two factor clinical trial, in which treatment A has three levels (A1, A2, and A3) and treatment B has four levels (B1, B2, B3, and B4). Drug company also tells you that the treatment combination A3:B4 has undesirable side effects so we ignore this design point. 

Using dummy coding with A1 and B1 as the baseline levels, find the matrix $C$ with each row a unique design point.

**Sol.**

In [76]:
treatment_combination = ["A1:B1", "A2:B1", "A3:B1", "A1:B2", "A2:B2", 
                         "A3:B2", "A1:B3", "A2:B3", "A3:B3", "A1:B4", "A2:B4"]
A2 = [0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1]
A3 = [0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0]
B2 = [0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0]
B3 = [0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0]
B4 = [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]

df = DataFrame(treatment_combination = treatment_combination, intcept = ones(11),
               A2 = A2, A3 = A3, B2 = B2, B3 = B3, B4 = B4)
df


Row,treatment_combination,intcept,A2,A3,B2,B3,B4
,String,Float64,Int64,Int64,Int64,Int64,Int64
1,A1:B1,1.0,0,0,0,0,0
2,A2:B1,1.0,1,0,0,0,0
3,A3:B1,1.0,0,1,0,0,0
4,A1:B2,1.0,0,0,1,0,0
5,A2:B2,1.0,1,0,1,0,0
6,A3:B2,1.0,0,1,1,0,0
7,A1:B3,1.0,0,0,0,1,0
8,A2:B3,1.0,1,0,0,1,0
9,A3:B3,1.0,0,1,0,1,0


## Q2 (30 pts) Find approximate optimal designs

Using semidefinite programming (SDP) software to find the approximate D-, E-, and A-optimal designs for this clinical trial.

Hint: This is what I got, which may or may not be correct.

```
Approximate Optimal Design
┌───────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┐
│ design_pt │   D_opt │   E_opt │   A_opt │ D_opt_n │ E_opt_n │ A_opt_n │
│    String │ Float64 │ Float64 │ Float64 │ Float64 │ Float64 │ Float64 │
├───────────┼─────────┼─────────┼─────────┼─────────┼─────────┼─────────┤
│      A1B1 │   0.082 │   0.272 │   0.200 │       8 │      27 │      20 │
│      A2B1 │   0.082 │   0.152 │   0.101 │       8 │      15 │      10 │
│      A3B1 │   0.097 │   0.114 │   0.104 │      10 │      11 │      10 │
│      A1B2 │   0.082 │   0.057 │   0.086 │       8 │       6 │       9 │
│      A2B2 │   0.082 │   0.039 │   0.051 │       8 │       4 │       5 │
│      A3B2 │   0.097 │   0.057 │   0.068 │      10 │       6 │       7 │
│      A1B3 │   0.082 │   0.057 │   0.086 │       8 │       6 │       9 │
│      A2B3 │   0.082 │   0.039 │   0.051 │       8 │       4 │       5 │
│      A3B3 │   0.097 │   0.057 │   0.068 │      10 │       6 │       7 │
│      A1B4 │   0.109 │   0.081 │   0.106 │      11 │       8 │      11 │
│      A2B4 │   0.109 │   0.073 │   0.080 │      11 │       7 │       8 │
└───────────┴─────────┴─────────┴─────────┴─────────┴─────────┴─────────┘
```

**Sol.**

In [77]:
X = hcat(ones(11), A2, A3, B2, B3, B4)
# define problem
P = Variable(size(X, 1))
# D opt
D_problem = minimize(-logdet(quadform(X, diagm(P)))) # objective
D_problem.constraints += sum(P) == 1; # constraint
D_problem.constraints += P >= 0; # constraint
# define solver
const MOI = MathOptInterface
solver = Mosek.Optimizer()
MOI.set(solver, MOI.RawOptimizerAttribute("LOG"), 0)
solve!(D_problem, solver)
# Check the status, optimal value, and minimizer of the problem
D_problem.status, D_problem.optval, P.value

(MathOptInterface.OPTIMAL, 8.98683156359766, [0.08194043080515269; 0.08195446134009557; … ; 0.10891345308189594; 0.1089056858612344;;])

In [78]:
# save number
D_opt_n = round.(P.value .* 100)

11×1 Matrix{Float64}:
  8.0
  8.0
 10.0
  8.0
  8.0
 10.0
  8.0
  8.0
 10.0
 11.0
 11.0

In [79]:
# E opt
E_problem = minimize(-eigmin(quadform(X, diagm(P)))) # objective
E_problem.constraints += sum(P) == 1; # constraint
E_problem.constraints += P >= 0; # constraint
# define solver
const MOI = MathOptInterface
solver = Mosek.Optimizer()
MOI.set(solver, MOI.RawOptimizerAttribute("LOG"), 0)
solve!(E_problem, solver)
# Check the status, optimal value, and minimizer of the problem
E_problem.status, E_problem.optval, P.value

(MathOptInterface.OPTIMAL, -0.076923076725408, [0.27116845314152616; 0.15605925265960144; … ; 0.0810948952008921; 0.07274779284884167;;])

In [80]:
# save number
E_opt_n = round.(P.value .* 100)

11×1 Matrix{Float64}:
 27.0
 16.0
 11.0
  6.0
  4.0
  6.0
  6.0
  4.0
  6.0
  8.0
  7.0

In [ ]:
# E opt
E_problem = minimize(-eigmin(quadform(X, diagm(P)))) # objective
E_problem.constraints += sum(P) == 1; # constraint
E_problem.constraints += P >= 0; # constraint

## Q3 (30 pts) Find exact optimal designs

Using mixed-integer semidefinite programming (SDP) software to find the exact D-, E-, and A-optimal designs for this clinical trial.

Hint: This is what I got. Apparently I haven't got the E-optimal design right yet.

```
Exact Optimal Design
┌───────────┬───────┬───────┬───────┐
│ design_pt │ D_opt │ E_opt │ A_opt │
│    String │ Int64 │ Int64 │ Int64 │
├───────────┼───────┼───────┼───────┤
│      A1B1 │     8 │    90 │    20 │
│      A2B1 │     8 │     1 │    10 │
│      A3B1 │    10 │     1 │    10 │
│      A1B2 │     8 │     1 │     9 │
│      A2B2 │     8 │     1 │     5 │
│      A3B2 │    10 │     1 │     7 │
│      A1B3 │     8 │     1 │     9 │
│      A2B3 │     8 │     1 │     5 │
│      A3B3 │    10 │     1 │     7 │
│      A1B4 │    11 │     1 │    10 │
│      A2B4 │    11 │     1 │     8 │
└───────────┴───────┴───────┴───────┘
```

## Q4 (30 bonus points) Optimal design with nuisance parameters

Suppose the regression coefficients of linear model $\boldsymbol{\beta}$ is partitioned as $\boldsymbol{\beta} = (\boldsymbol{\beta}_0^T, \boldsymbol{\beta}_1^T)^T$, where $\boldsymbol{\beta}_0$ are nuisance parameters and $\boldsymbol{\beta}_1$ are parameters of primary interest. Given an approximate design $\mathbf{p} = (p_1, \ldots, p_m)$, let the information matrix be partitioned accordingly
$$
\mathbf{I}(\mathbf{p}) = \sum_{i=1}^m p_i \mathbf{x}_i \mathbf{x}_i^T =  \begin{pmatrix}
\mathbf{I}_{00} & \mathbf{I}_{01} \\
\mathbf{I}_{10} & \mathbf{I}_{11}
\end{pmatrix}.
$$
Then the information matrix for $\boldsymbol{\beta}_1$ adjusted for nuisance parameter $\boldsymbol{\beta}_0$ is
$$
\mathbf{I}_{1 \mid 0}(\mathbf{p}) = \mathbf{I}_{11} - \mathbf{I}_{10} \mathbf{I}_{00}^{-1} \mathbf{I}_{01}.
$$

Revisiting the 3x4 factorial design problem in Q1, suppose the drug company only cares about the estimation of A treatment effects. Find the approximate D-, E-, and A-optimal designs.